# RTTS Training & Jetson Deployment (Colab)
This notebook automates RTTS dataset preparation, YOLOX-S training on a Colab GPU runtime, and packaging of artifacts for Jetson Nano inference. Run the cells sequentially after switching the Colab runtime to **GPU** (`Runtime -> Change runtime type -> GPU`).

## 1. Configure Colab GPU Environment
The following cells verify GPU availability, install system dependencies, and clone this repository into `/content/Object-Detection-in-Hazy-and-Foggy-Conditions-on-NVIDIA-Jetson-Nano`.

In [ ]:
# Verify that Colab attached a GPU runtime
!nvidia-smi

In [ ]:
# Clone the repository and install Python dependencies
import os
from pathlib import Path

REPO_URL = "https://github.com/wassihaiderkabir/Object-Detection-in-Hazy-and-Foggy-Conditions-on-NVIDIA-Jetson-Nano.git"
REPO_DIR = Path("/content/Object-Detection-in-Hazy-and-Foggy-Conditions-on-NVIDIA-Jetson-Nano")
if not REPO_DIR.exists():
    !git clone {REPO_URL} {REPO_DIR}

%cd {REPO_DIR}
!python -m pip install --upgrade pip wheel
!pip install -r requirements.txt

## 2. Prepare Dataset
This section downloads the RTTS subset of RESIDE into `/content/data/RTTS`, verifies the Pascal VOC-style splits, and runs the RTTS sanity checker.

In [ ]:
# Download and unpack RTTS into /content/data/RTTS
from pathlib import Path
import shutil

DATA_ROOT = Path("/content/data")
RTTS_DIR = DATA_ROOT / "RTTS"
DATA_ROOT.mkdir(parents=True, exist_ok=True)
if not RTTS_DIR.exists():
    !wget -q https://residedata.obs.cn-north-4.myhuaweicloud.com/RTTS.zip -O {DATA_ROOT / "RTTS.zip"}
    !unzip -q {DATA_ROOT / "RTTS.zip"} -d {DATA_ROOT}
    !rm {DATA_ROOT / "RTTS.zip"}
print(f"RTTS dataset ready at {RTTS_DIR}")

In [ ]:
# Run the RTTS sanity checker to confirm class distribution
import os
os.environ["YOLOX_DATADIR"] = str(DATA_ROOT)
%cd {REPO_DIR}
!python src/Jetson_src/temp_inspect_rtts.py --refresh --limit 50

### Results Directory Setup
Create a unified `/content/.../results` tree so plots, weights, and inference artifacts end up in one place for easy download.

In [ ]:
# Create shared results directories and handy path helpers
RESULTS_DIR = REPO_DIR / "results"
PLOTS_DIR = RESULTS_DIR / "plots"
WEIGHTS_DIR = RESULTS_DIR / "weights"
INFER_DIR = RESULTS_DIR / "inference_samples"
RESULTS_DIR.mkdir(exist_ok=True)
for path in (PLOTS_DIR, WEIGHTS_DIR, INFER_DIR):
    path.mkdir(parents=True, exist_ok=True)

EXP_NAME = "rtts_yolox_s"
OUTPUT_DIR = REPO_DIR / "YOLOX_outputs" / EXP_NAME
BEST_CKPT = OUTPUT_DIR / "best_ckpt.pth"
LATEST_CKPT = OUTPUT_DIR / "latest_ckpt.pth"
TENSORBOARD_DIR = OUTPUT_DIR / "tensorboard"
print(f"Results will be stored under {RESULTS_DIR}")
print(f"TensorBoard logs expected in {TENSORBOARD_DIR}")

## 3. Build Model Architecture
YOLOX supplies the RTTS-specific experiment in `src/exps/example/custom/rtts_yolox_s.py`. The next cell prints key hyperparameters so you can tweak depth/width, input resolution, or augmentation knobs before training.

In [ ]:
# Display the core experiment settings for YOLOX-S on RTTS
from pathlib import Path
exp_path = REPO_DIR / "src" / "exps" / "example" / "custom" / "rtts_yolox_s.py"
print(exp_path.read_text())

## 4. Train Model with GPU Acceleration
We launch `src/Jetson_src/train_rtts.py`, which wires `YOLOX_DATADIR`, enables mixed precision, and saves checkpoints under `YOLOX_outputs/rtts_yolox_s/`. Adjust `--batch-size`, `--max-epochs`, or `--amp/--no-amp` to fit your GPU budget.

In [ ]:
# Launch fine-tuning on RTTS
%cd {REPO_DIR}
!python src/Jetson_src/train_rtts.py \
    --colab \
    --datadir /content/data \
    --exp src/exps/example/custom/rtts_yolox_s.py \
    --ckpt src/yolox/weights/yolox_s.pth \
    --exp-name {EXP_NAME} \
    --batch-size 16

### Persist Checkpoints to `results/weights`
Mirror the best and latest checkpoints into the shared results directory so they can be zipped or downloaded later.

In [ ]:
# Copy checkpoints into results/weights for convenient download
import shutil
weights_copied = 0
for ckpt in [BEST_CKPT, LATEST_CKPT]:
    if ckpt.exists():
        destination = WEIGHTS_DIR / ckpt.name
        shutil.copy2(ckpt, destination)
        weights_copied += 1
        print(f"Copied {ckpt.name} -> {destination}")
    else:
        print(f"[WARN] {ckpt} not found yet; re-run after training finishes.")
print(f"Total checkpoints mirrored: {weights_copied}")

## 5. Evaluate and Save Artifacts
After training completes, run evaluation on the validation split, export ONNX plus TensorRT-ready assets, and bundle everything for download to your workstation.

In [ ]:
# Evaluate best checkpoint on RTTS val split
!python src/tools/eval.py \
    -f src/exps/example/custom/rtts_yolox_s.py \
    -c {BEST_CKPT} \
    -b 1 -d 1 --conf 0.001 --fp16

In [ ]:
# Export ONNX and convert to TensorRT-friendly artifacts
EXPORT_DIR = REPO_DIR / "exports"
EXPORT_DIR.mkdir(exist_ok=True)
ONNX_PATH = EXPORT_DIR / "rtts_yolox_s.onnx"
!python src/tools/export_onnx.py \
    -f src/exps/example/custom/rtts_yolox_s.py \
    -c {BEST_CKPT} \
    --output-file {ONNX_PATH} \
    --input [640,640]

TRT_ENGINE = EXPORT_DIR / "rtts_yolox_s_fp16.trt"
!python src/tools/trt.py \
    -f src/exps/example/custom/rtts_yolox_s.py \
    -c {BEST_CKPT} \
    --trt_fp16 --device 0 \
    --output-name {TRT_ENGINE}

### Visualize Training & Validation Curves
Use the TensorBoard event logs to chart training losses, learning rate, and validation AP. All plots are saved under `results/plots`.

In [ ]:
# Plot training/validation curves from TensorBoard logs
import matplotlib.pyplot as plt
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

plt.style.use("seaborn-v0_8-darkgrid")

if not TENSORBOARD_DIR.exists():
    raise FileNotFoundError(f"TensorBoard directory not found: {TENSORBOARD_DIR}")

accumulator = EventAccumulator(str(TENSORBOARD_DIR))
accumulator.Reload()
scalar_tags = set(accumulator.Tags().get("scalars", []))


def load_series(tag: str) -> pd.DataFrame:
    events = accumulator.Scalars(tag)
    return pd.DataFrame({
        "step": [event.step for event in events],
        "value": [event.value for event in events],
        "tag": tag,
    })

# Training losses
train_tags = [
    tag for tag in (
        "train/total_loss",
        "train/iou_loss",
        "train/conf_loss",
        "train/cls_loss",
        "train/lr",
    ) if tag in scalar_tags
]

figures = []
if train_tags:
    fig, ax = plt.subplots(figsize=(8, 4))
    for tag in train_tags:
        series = load_series(tag)
        label = tag.split("/", 1)[1]
        ax.plot(series["step"], series["value"], label=label)
    ax.set_xlabel("iteration")
    ax.set_ylabel("value")
    ax.set_title("Training losses & LR")
    ax.legend(loc="upper right")
    fig.tight_layout()
    train_plot = PLOTS_DIR / "train_curves.png"
    fig.savefig(train_plot, dpi=200)
    figures.append(train_plot)
    plt.show()
else:
    print("No train/* scalars in TensorBoard logs (did training finish?).")

# Validation AP curves
val_tags = [tag for tag in ("val/COCOAP50", "val/COCOAP50_95") if tag in scalar_tags]
if val_tags:
    fig, ax = plt.subplots(figsize=(8, 4))
    for tag in val_tags:
        series = load_series(tag)
        label = tag.split("/", 1)[1]
        ax.plot(series["step"], series["value"], marker="o", label=label)
    ax.set_xlabel("epoch")
    ax.set_ylabel("AP")
    ax.set_title("Validation AP progression")
    ax.legend(loc="lower right")
    fig.tight_layout()
    val_plot = PLOTS_DIR / "val_ap.png"
    fig.savefig(val_plot, dpi=200)
    figures.append(val_plot)
    plt.show()
else:
    print("No val/* scalars found; ensure eval_interval > 0 during training.")

print("Stored plots:")
for fig_path in figures:
    print(" -", fig_path)


### Local Inference Smoke Test
Run `tools/demo.py` on a representative RTTS image, copy the rendered result into `results/inference_samples`, and display it inline to verify the checkpoint.

In [ ]:
# Run a quick image inference and archive the rendered output
import shutil
from pathlib import Path
from IPython.display import Image as IPyImage, display

jpeg_root = RTTS_DIR / "JPEGImages"
sample_candidates = sorted(list(jpeg_root.glob("*.png"))) or sorted(list(jpeg_root.glob("*.jpg")))
if not sample_candidates:
    raise FileNotFoundError(f"No images found inside {jpeg_root}")
SAMPLE_IMAGE = sample_candidates[0]
print(f"Using sample image: {SAMPLE_IMAGE}")

vis_root = OUTPUT_DIR / "vis_res"
!python src/tools/demo.py image \
    -f src/exps/example/custom/rtts_yolox_s.py \
    -c {BEST_CKPT} \
    --path {SAMPLE_IMAGE} \
    --conf 0.25 --nms 0.45 \
    --device gpu --fp16 --save_result

if not vis_root.exists():
    raise FileNotFoundError(f"No visualization directory found at {vis_root}")
latest_folder = max(vis_root.iterdir(), key=lambda d: d.stat().st_mtime)
rendered = list(latest_folder.glob(SAMPLE_IMAGE.name)) or list(latest_folder.glob("*"))
if not rendered:
    raise FileNotFoundError(f"No rendered files found under {latest_folder}")
render_path = rendered[0]
target = INFER_DIR / render_path.name
shutil.copy2(render_path, target)
print(f"Copied rendered image to {target}")
display(IPyImage(filename=target))

### Confusion Matrix on Validation Split
Re-run evaluation inside this notebook, collect detections, and derive a confusion matrix (IoU ≥ 0.5) to spot which classes still confuse the model. Results are written to `results/plots/confusion_matrix.png`.

In [ ]:
# Build a confusion matrix by matching predictions to ground truth at IoU >= 0.5
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
from yolox.exp import get_exp

plt.style.use("seaborn-v0_8-darkgrid")

device = "cuda" if torch.cuda.is_available() else "cpu"
exp = get_exp(str(REPO_DIR / "src" / "exps" / "example" / "custom" / "rtts_yolox_s.py"), None)
model = exp.get_model()
ckpt = torch.load(BEST_CKPT, map_location="cpu")
model.load_state_dict(ckpt["model"], strict=False)
model.to(device).eval()

evaluator = exp.get_evaluator(batch_size=1, is_distributed=False)
(_, _, _), predictions = evaluator.evaluate(model, half=torch.cuda.is_available(), return_outputs=True)
dataset = evaluator.dataloader.dataset
class_names = list(getattr(dataset, "_classes", [f"class_{i}" for i in range(exp.num_classes)]))
labels = class_names + ["background"]
mat = np.zeros((len(labels), len(labels)), dtype=int)
iou_thresh = 0.5
max_images = min(len(predictions), 400)


def iou(box, boxes):
    if boxes.size == 0:
        return np.array([])
    box = np.expand_dims(box, axis=0)
    lt = np.maximum(box[..., :2], boxes[..., :2])
    rb = np.minimum(box[..., 2:], boxes[..., 2:])
    wh = np.clip(rb - lt, a_min=0, a_max=None)
    inter = wh[..., 0] * wh[..., 1]
    box_area = (box[..., 2] - box[..., 0]) * (box[..., 3] - box[..., 1])
    boxes_area = (boxes[..., 2] - boxes[..., 0]) * (boxes[..., 3] - boxes[..., 1])
    union = box_area + boxes_area - inter
    return inter / np.clip(union, a_min=1e-6, a_max=None)


def to_numpy(pred_tuple):
    if pred_tuple[0] is None:
        return np.zeros((0, 4)), np.zeros((0,), dtype=int), np.zeros((0,))
    boxes, cls, scores = pred_tuple
    return boxes.numpy(), cls.numpy().astype(int), scores.numpy()


processed = 0
for img_id in sorted(predictions.keys()):
    if processed >= max_images:
        break
    pred_boxes, pred_cls, pred_scores = to_numpy(predictions[img_id])
    gt = dataset.annotations[img_id][0] if len(dataset.annotations) > img_id else np.zeros((0, 5))
    gt_boxes = gt[:, :4] if gt.size else np.zeros((0, 4))
    gt_cls = gt[:, 4].astype(int) if gt.size else np.zeros((0,), dtype=int)

    order = np.argsort(-pred_scores)
    pred_boxes, pred_cls, pred_scores = pred_boxes[order], pred_cls[order], pred_scores[order]
    matched_gt = set()

    for p_box, p_cls in zip(pred_boxes, pred_cls):
        if gt_boxes.size:
            ious = iou(p_box, gt_boxes)
            best_idx = int(np.argmax(ious)) if ious.size else -1
            best_iou = ious[best_idx] if ious.size else 0.0
        else:
            best_idx, best_iou = -1, 0.0

        if best_idx >= 0 and best_iou >= iou_thresh and best_idx not in matched_gt:
            gt_label = gt_cls[best_idx]
            mat[gt_label, p_cls] += 1
            matched_gt.add(best_idx)
        else:
            mat[len(class_names), p_cls] += 1  # false positives counted in last row

    for gt_idx, gt_label in enumerate(gt_cls):
        if gt_idx not in matched_gt:
            mat[gt_label, len(class_names)] += 1  # misses counted in last column

    processed += 1

print(f"Confusion stats built from {processed} validation images")
cm_path = PLOTS_DIR / "confusion_matrix.png"
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(mat, cmap="Blues")
ax.set_xticks(range(len(labels)))
ax.set_yticks(range(len(labels)))
ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_yticklabels(labels)
ax.set_xlabel("Predicted class")
ax.set_ylabel("Ground truth class")
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
fig.tight_layout()
fig.savefig(cm_path, dpi=220)
plt.show()
print(f"Confusion matrix saved to {cm_path}")

## 6. Package Jetson Nano Inference Script
Jetson-friendly inference uses TensorRT plus lightweight preprocessing. The cell below emits `src/Jetson_src/jetson_nano_runner.py`, which loads the exported ONNX/TensorRT engine, runs warmup, and exposes CLI switches for images, video files, or live cameras.

In [ ]:
# Create the Jetson Nano inference helper script
from textwrap import dedent

script_path = REPO_DIR / "src" / "Jetson_src" / "jetson_nano_runner.py"
script_path.write_text(dedent('''#!/usr/bin/env python3
"""TensorRT-friendly inference helper tailored for Jetson Nano."""

import argparse
import os
import shutil
import subprocess
from pathlib import Path
from typing import List

ROOT = Path(__file__).resolve().parents[2]
DEFAULT_EXP = ROOT / "src" / "exps" / "example" / "custom" / "rtts_yolox_s.py"
DEFAULT_EXP_NAME = "rtts_yolox_s"
DEFAULT_CKPT = ROOT / "YOLOX_outputs" / DEFAULT_EXP_NAME / "best_ckpt.pth"
DEFAULT_TRT = ROOT / "YOLOX_outputs" / DEFAULT_EXP_NAME / "model_trt.pth"
DEMO_SCRIPT = ROOT / "src" / "tools" / "demo.py"


def _run(cmd: List[str]) -> None:
    print("[jetson_nano_runner]", " ".join(cmd))
    subprocess.run(cmd, check=True, cwd=ROOT)


def _ensure_trt(args: argparse.Namespace) -> None:
    if args.trt_file.exists() and not args.rebuild:
        print(f"[jetson_nano_runner] Reusing TensorRT weights at {args.trt_file}")
        return

    build_cmd = [
        "python3",
        "src/tools/trt.py",
        "-f",
        str(args.exp),
        "-c",
        str(args.ckpt),
        "-expn",
        args.exp_name,
    ]
    _run(build_cmd)

    built_file = ROOT / "YOLOX_outputs" / args.exp_name / "model_trt.pth"
    if not built_file.exists():
        raise FileNotFoundError(
            f"TensorRT conversion did not produce {built_file}. Check tools/trt.py logs."
        )
    args.trt_file.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy2(built_file, args.trt_file)
    print(f"[jetson_nano_runner] Copied TensorRT weights to {args.trt_file}")


def _launch_demo(args: argparse.Namespace) -> None:
    cmd = [
        "python3",
        str(DEMO_SCRIPT),
        args.mode,
        "-f",
        str(args.exp),
        "--device",
        "gpu",
        "--conf",
        str(args.conf),
        "--nms",
        str(args.nms),
        "--trt",
        "--trt-file",
        str(args.trt_file),
        "--save_result",
    ]
    if args.fp16:
        cmd.append("--fp16")
    if args.tsize:
        cmd.extend(["--tsize", str(args.tsize)])

    if args.mode in {"image", "video"}:
        cmd.extend(["--path", str(args.input)])
    else:
        cmd.extend(["--camid", str(args.cam_id)])

    _run(cmd)


def main() -> None:
    parser = argparse.ArgumentParser(description=__doc__)
    parser.add_argument("--exp", type=Path, default=DEFAULT_EXP, help="Experiment file")
    parser.add_argument("--exp-name", default=DEFAULT_EXP_NAME, help="Experiment/output name")
    parser.add_argument("--ckpt", type=Path, default=DEFAULT_CKPT, help="Checkpoint path")
    parser.add_argument("--trt-file", type=Path, default=DEFAULT_TRT, help="TensorRT weight file")
    parser.add_argument("--mode", choices=["image", "video", "webcam"], default="image")
    parser.add_argument("--input", type=Path,
                        default=ROOT / "data" / "RTTS" / "JPEGImages" / "AM_Bing_211.png")
    parser.add_argument("--cam-id", type=int, default=0)
    parser.add_argument("--conf", type=float, default=0.25)
    parser.add_argument("--nms", type=float, default=0.45)
    parser.add_argument("--tsize", type=int, default=640, help="Square inference size")
    parser.add_argument("--datadir", type=Path, default=ROOT / "data")
    parser.add_argument("--rebuild", action="store_true", help="Force TensorRT rebuild")
    parser.add_argument("--skip-build", action="store_true", help="Skip TensorRT conversion")
    parser.add_argument("--fp16", action="store_true", help="Enable FP16 inference")
    args = parser.parse_args()

    os.environ.setdefault("YOLOX_DATADIR", str(args.datadir.resolve()))

    if not args.skip_build:
        _ensure_trt(args)

    _launch_demo(args)


if __name__ == "__main__":
    main()
'''))
script_path.chmod(0o755)
print(f"Jetson Nano runner created at {script_path}")

In [ ]:
# Bundle key artifacts (results, exports, scripts) into a single zip for download
import zipfile
from pathlib import Path

BUNDLE_NAME = "rtts_yolox_s_artifacts.zip"
BUNDLE_PATH = RESULTS_DIR / BUNDLE_NAME
if BUNDLE_PATH.exists():
    BUNDLE_PATH.unlink()

paths_to_package = [
    RESULTS_DIR,
    EXPORT_DIR,
    REPO_DIR / "src" / "exps" / "example" / "custom" / "rtts_yolox_s.py",
    REPO_DIR / "src" / "Jetson_src" / "jetson_nano_runner.py",
]

files_to_add = []
for path in paths_to_package:
    path = Path(path)
    if not path.exists():
        print(f"[WARN] Skipping missing path: {path}")
        continue
    if path.is_dir():
        files_to_add.extend([p for p in path.rglob("*") if p.is_file()])
    else:
        files_to_add.append(path)

with zipfile.ZipFile(BUNDLE_PATH, "w", zipfile.ZIP_DEFLATED) as zf:
    for file_path in files_to_add:
        if file_path.resolve() == BUNDLE_PATH.resolve():
            continue
        arcname = file_path.relative_to(REPO_DIR)
        zf.write(file_path, arcname)

print(f"Bundle ready at {BUNDLE_PATH} (contains {len(files_to_add)} files)")

## 7. Document Jetson Nano Setup & Deployment Steps
1. **Flash & update Jetson Nano**: Install JetPack 5.1+ (ships with CUDA, cuDNN, TensorRT). Run `sudo nvpmodel -m 0 && sudo jetson_clocks` to unlock the 10 W power profile.
2. **Install dependencies** (already bundled with JetPack, just add extras): `sudo apt install python3-pip libopencv-dev && pip3 install --upgrade pip` followed by `pip3 install -r requirements.txt --extra-index-url https://pypi.ngc.nvidia.com` inside the repo directory.
3. **Copy artifacts** produced by this notebook: `scp rtts_yolox_s_artifacts.zip jetson@nano.local:~/haze-yolox/ && unzip rtts_yolox_s_artifacts.zip`.
4. **Verify datasets**: Place RTTS under `~/haze-yolox/data/RTTS` (same layout as on Colab) and set `YOLOX_DATADIR=/home/jetson/haze-yolox/data`.
5. **Optional TensorRT rebuild**: `python3 src/tools/trt.py -f src/exps/example/custom/rtts_yolox_s.py -c YOLOX_outputs/rtts_yolox_s/best_ckpt.pth --trt_fp16 --output-name exports/rtts_yolox_s_fp16.trt`.
6. **Run inference** with the helper script: `python3 src/Jetson_src/jetson_nano_runner.py --mode image --input data/RTTS/JPEGImages/AM_Bing_211.png --conf 0.25 --nms 0.45`. Switch to `--mode video --input foggy.mp4` or `--mode webcam --cam-id 0` as needed.
7. **Monitor performance**: Use `tegrastats` to watch GPU load/temperature and adjust resolution via `--img-size 512 512` if FPS dips below your target.